### prep

In [100]:
import pickle
import pandas as pd
import numpy as np

In [132]:
with open('df1.pkl', 'rb') as f:
    data = pickle.load(f)
df = pd.DataFrame(data)
df.head()

,bowler,batsman,wicket,over,outcome
0,Mortaza,Warner,0,1,0
1,Mortaza,Warner,0,1,0
2,Mortaza,Warner,0,1,1
3,Mortaza,Watson,0,1,0
4,Mortaza,Watson,0,1,1


In [133]:
X = np.load("X_mat.npy")

In [134]:
n_mat = X.sum(axis=3, keepdims=True)

In [135]:
n_mat.shape

(434, 20, 10, 1)

In [136]:
emp_prob = X / (n_mat+0.001)
# emp_prob.shape
np.max(emp_prob)

0.9998889012331964

In [137]:
## Delta theorem multipliers
inv_prob = np.array([(1-p)/(p+0.001) for p in emp_prob])
inv_prob = inv_prob/(n_mat+0.001)
# inv_prob.shape

### alpha matrix

In [138]:
alpha=[]
v_inv=[]
for i in range(emp_prob.shape[0]):
    alpha_i=[]
    v_inv_i=[]
    for o in range(19):
        val = emp_prob[i,o+1]/(emp_prob[i,o]+0.001)
        alpha_i.append(val)
        v_i = np.square(val)*(inv_prob[i,o+1]+inv_prob[i,o]) 
        v_inv_i.append([1/(np.sqrt(x)+0.001) for x in v_i])
        # v_i_root = np.sqrt(v_i)
        # print(v_i_root.shape)
        # v_inv_i.append(v_i_root)
    alpha.append(alpha_i)
    v_inv.append(v_inv_i)
    
alpha = np.array(alpha)
v_inv = np.array(v_inv)
v_inv.shape

(434, 19, 10, 8)

In [139]:
den = np.sum(v_inv, axis=0)
num = np.sum(v_inv*alpha, axis=0)
o_trans = num/den

### beta matrix

In [140]:
beta=[]
w_inv=[]
for i in range(emp_prob.shape[0]):
    beta_o=[]
    y_inv_o=[]
    for o in range(20):
        beta_i=[]
        v_inv_i=[]
        for w in range(9):
            val = emp_prob[i,o,w+1]/(emp_prob[i,o,w]+0.001)
            beta_i.append(val)
            v_i = np.square(val)*(inv_prob[i,o,w+1]+inv_prob[i,o,w]) 
            v_inv_i.append([1/(np.sqrt(x)+0.001) for x in v_i])
            # print(val.shape)
        beta_o.append(beta_i)
        y_inv_o.append(v_inv_i)
    beta.append(beta_o)
    w_inv.append(y_inv_o)
    
beta = np.array(beta)
w_inv = np.array(w_inv)
w_inv.shape 

(434, 20, 9, 8)

In [141]:
w_trans = np.sum(w_inv*beta, axis=0)/np.sum(w_inv, axis=0)
w_trans.shape

(20, 9, 8)

### tau calc

In [142]:
multiplier_mat = np.ones((2,20,10,8))
for o in range(1,20):
    multiplier_mat[0,o] = multiplier_mat[0,o-1]*o_trans[o-1]
for w in range(1,10):
    multiplier_mat[1,:,w] = multiplier_mat[1,:,w-1]*w_trans[:,w-1]

tau_mat = multiplier_mat[0]*multiplier_mat[1]
tau_mat = tau_mat/tau_mat[6,0] # tau_mat[6,0,:]=1

### parameter estimation

In [143]:
emp_prob.shape

(434, 20, 10, 8)

In [144]:
np.sum(emp_prob[0,0,0])

0.9999922481221075